In [1]:
import numpy as np
import sys
sys.path.append('../src/PrimeFactorization')
from PrimeFactorization import primeFactorize
import copy
from aks import aks
import itertools
import functools
import math

In [2]:
@functools.lru_cache(maxsize=1000)
def is_prime(n, n_iter=1000):
    d = 2
    while d * d <= n:
        if n % d == 0:
            return False
        d += 1
    return n > 1

In [3]:
# https://rosettacode.org/wiki/Miller%E2%80%93Rabin_primality_test
def _try_composite(a, d, n, s):
    if pow(a, d, n) == 1:
        return False
    for i in range(s):
        if pow(a, 2**i * d, n) == n-1:
            return False
    return True # n  is definitely composite
@functools.lru_cache(maxsize=1000)
def is_prime(n, _precision_for_huge_n=16):
    n=int(n)
    if n < 2: return False
    if n <= _known_primes[-1]:
        if n in _known_primes:
            return True
    for p in _known_primes:
        if n % p == 0:
            return False
    d, s = n - 1, 0
    while not d % 2:
        d, s = d >> 1, s + 1
    # Returns exact according to http://primes.utm.edu/prove/prove2_3.html
    if n < 1373653: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3))
    if n < 25326001: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5))
    if n < 118670087467: 
        if n == 3215031751: 
            return False
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7))
    if n < 2152302898747:
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11))
    if n < 3474749660383: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11, 13))
    if n < 341550071728321:
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11, 13, 17))
    # otherwise
    return not any(_try_composite(a, d, n, s) 
                for a in _known_primes[:_precision_for_huge_n])
_known_primes = [2, 3]
_known_primes += [x for x in range(5, 1000, 2) if is_prime(x)]
 

In [65]:
def try_find_factor(number, scale=2, offset=0, verbose=False):
    base = np.ceil(math.sqrt(number))
    if verbose:
        print('base, scale, offset: ', base, scale, offset)
    shifted = base + offset
    scaled = scale*shifted
    
    if verbose:
        print('S, S\': ', shifted, scaled)
        print("shifted squared, scaled base squared: ",shifted**2, scaled**2)
    
    shifted_squared_mod_n = shifted**2 % number
#     print('WARNING: EXPERIMENTAL ENABLED!!!!')
    scaled_shifted_squared_mod_n = scaled**2 % number
#     scaled_shifted_squared_mod_n = shifted_squared_mod_n * scale**2
    
    if verbose:
        print('R, R\': ', shifted_squared_mod_n, scaled_shifted_squared_mod_n)
    
    
    first_term = shifted * scaled
    second_term = math.sqrt(shifted_squared_mod_n * scaled_shifted_squared_mod_n)
    
    if verbose:
        print('X, Y: ', first_term, second_term)
        print('X**2 % N: ', first_term**2 % number)
        print('Y**2 % N: ', second_term**2 % number)
    
    plus_term = first_term + second_term
    minus_term = abs(first_term - second_term)
    
    if verbose:
        print('f+, f-: ', plus_term, minus_term)
        # print('%.18f'%plus_term)
    
    gcd_1 = np.gcd(int(plus_term), number)
    gcd_2 = np.gcd(int(minus_term), number)
    if verbose:
        print('gcds: ', gcd_1, gcd_2)
        print('='*100)
    
    return gcd_1, gcd_2

In [66]:
def valid_scale_offset(number, scale, offset):
    base = np.ceil(math.sqrt(number))
    shifted = (base + offset)**2
    shifted_mod = shifted % number
    scaled = shifted_mod * scale**2
    return scaled // number == 0

In [72]:
number=12345
scale=5
offset=0
valid_scale = valid_scale_offset(number,scale,offset)
print(valid_scale)
if valid_scale or True:
    try_find_factor(number, scale, offset, verbose=True)

True
base, scale, offset:  112.0 5 0
S, S':  112.0 560.0
shifted squared, scaled base squared:  12544.0 313600.0
R, R':  199.0 4975.0
X, Y:  62720.0 995.0
X**2 % N:  2425.0
Y**2 % N:  2425.0
f+, f-:  63715.0 61725.0
gcds:  5 12345


In [52]:
12345*2

24690

In [62]:
def try_factor_term(number, max_offset, max_scale):
    if max_scale > 0:
        valid_scales = np.arange(2, min(max_scale, number))
    else:
        valid_scales = np.arange(2, number)
        
    if max_offset > 0:
        valid_offsets = np.arange(min(max_offset, number))
    else:
        valid_offsets = np.arange(number)
    
    valid_scale_offsets = itertools.product(valid_scales, valid_offsets)
    
    base = np.ceil(math.sqrt(number))
    if base**2==number:
        return int(base)
#     shifted = (base + valid_scale_offsets[:,1])**2
#     shifted_mod = shifted % number
#     scaled = shifted_mod * valid_scale_offsets[:,0]**2
#     invalid = scaled // number > 0
    
#     valid_scale_offsets = valid_scale_offsets[~invalid]
    
    
    for scale, offset in valid_scale_offsets:
#         if not valid_scale_offset(number, scale, offset):
#             continue
        gcd1, gcd2 = try_find_factor(number, scale, offset)
#         print(gcd1, gcd2)
        if 1 < gcd1 < number:
            print('Success!: number, base, scale, offset, factor: ', number, base, scale, offset, gcd1)
            print('Was it valid?: ', valid_scale_offset(number, scale, offset))
            #print('='*100)
            return gcd1
        if 1 < gcd2 < number:
            print('Success!: number, base, scale, offset, factor: ', number, base, scale, offset, gcd2)
            print('Was it valid?: ', valid_scale_offset(number, scale, offset))
            #print('='*100)
            return gcd2
    raise ValueError('Algorithm Failed to Converge :( Try a larger scale or offset')

In [63]:
def factor(number, max_offset, max_scale):
    factors = [number]
#     print(factors)
    while not all([is_prime(n) for n in factors]):
        new_factors = []
        for f in factors:
            if is_prime(f):
                new_factors.append(f)
                continue
            new_factor = try_factor_term(f, max_offset, max_scale)
            new_factors.append(new_factor)
            new_factors.append(f//new_factor)
        factors = new_factors
    return sorted(factors)

In [64]:
%%time
factor(123456789123456789123456789123456789, 10000, 100)

Success!: number, base, scale, offset, factor:  123456789123456789123456789123456789 3.513641830401283e+17 2 32 19
Was it valid?:  True
Success!: number, base, scale, offset, factor:  6497725743339831006497725743339831 8.060847190798142e+16 2 0 3
Was it valid?:  True
Success!: number, base, scale, offset, factor:  2165908581113277002165908581113277 4.6539322955037464e+16 2 4 77
Was it valid?:  True
Success!: number, base, scale, offset, factor:  77 9.0 2 2 11
Was it valid?:  False
Success!: number, base, scale, offset, factor:  28128682871601000028128682871601 5303648071997330.0 2 1 39
Was it valid?:  True
Success!: number, base, scale, offset, factor:  39 7.0 2 2 3
Was it valid?:  True
Success!: number, base, scale, offset, factor:  721248278759000000721248278759 849263374200843.0 2 44 101
Was it valid?:  True
Success!: number, base, scale, offset, factor:  7141072066920792086348992859 84504864161307.0 2 1164 3607
Was it valid?:  True
Success!: number, base, scale, offset, factor:  19

[3, 3, 7, 11, 13, 19, 101, 3607, 3803, 9901, 52579, 999999000001]

In [43]:
%%time
factor(123456789123456789123456789, 1000, 100)

Wall time: 31 ms


[3, 3, 3, 757, 3607, 3803, 440334654777631]

In [ ]:
%%time
primeFactorize(123456789)

In [ ]:
np.log(123456789)/np.log(2)